In [1]:
import pandas as pd
import os

In [2]:
os.chdir("/home/mosouwer/Documents/fake-news")

In [3]:
df=pd.read_csv("train.csv")

In [4]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [7]:
#drop na
df=df.dropna()

In [9]:
## get independant feature
X=df.drop("label",axis=1)

In [10]:
## Get dependant features
Y=df['label']

In [11]:
X.shape

(18285, 4)

In [12]:
Y.shape

(18285,)

In [14]:
import tensorflow as tf
tf.__version__

'2.3.0'

In [15]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

## One hot Representation

In [16]:
msg=X.copy()

In [18]:
msg.reset_index(inplace=True)

In [19]:
import nltk
import re
from nltk.corpus import stopwords

In [21]:
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [22]:
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
import re

lemma=WordNetLemmatizer()
corpus=[]
for i in range(len(msg)):
    review=re.sub('[^a-zA-Z]', " ",msg['title'][i])
    review=review.lower()
    review=review.split()
    review=[lemma.lemmatize(word) for word in review if not word in set(stopwords.words('english')) ]
    review=' '.join(review)
    corpus.append(review)

In [24]:
voc_size=10000

In [26]:
onehot_reps=[one_hot(words,voc_size)for words in corpus]
print(onehot_reps)

[[9283, 9214, 4060, 5956, 4359, 7249, 4077, 4541, 1415, 3706], [1819, 5715, 4867, 938, 5544, 2756, 6984], [611, 2360, 1841, 6175], [320, 8014, 3265, 3154, 6178, 6451], [3744, 5544, 5675, 5471, 7055, 813, 5544, 4431, 9055, 4872], [8920, 4725, 9501, 4505, 1549, 3226, 8623, 1254, 6100, 9382, 9952, 688, 9291, 7639, 6984], [9289, 5398, 6637, 809, 6610, 6314, 5231, 9770, 1001, 3266, 5958], [228, 6879, 7197, 4605, 7074, 1023, 3226, 4650, 1001, 3266, 5958], [1229, 5516, 8569, 8645, 7167, 1316, 5593, 2090, 3226, 5096], [8240, 5058, 6151, 4957, 5744, 4283, 692, 3614], [6606, 5991, 1984, 2635, 8988, 1242, 4673, 6666, 7191, 4516, 8686], [3154, 3134, 4359, 8537, 3226, 7074], [516, 9725, 4906, 4446, 2046, 9545, 9206, 5385, 4572], [3918, 4324, 3535, 3915, 1401, 3494, 9757, 1001, 3266, 5958], [5070, 8269, 379, 9527, 8846, 1001, 3266, 5958], [7821, 1240, 7977, 1545, 819, 288, 7554, 7858, 9631, 5046], [579, 3437, 5715], [7747, 5818, 1384, 8364, 3226, 7885, 4663, 6984], [5389, 4561, 4867, 3745, 3123, 439

## Pad Sequences for Embedding Layer

In [28]:
embedded_docs=pad_sequences(onehot_reps,padding="pre",maxlen=20)
print(embedded_docs[0])

[   0    0    0    0    0    0    0    0    0    0 9283 9214 4060 5956
 4359 7249 4077 4541 1415 3706]


In [29]:
#creating model and adding layers
vec_features=40
model=Sequential()
model.add(Embedding(voc_size,vec_features,input_length=20))
model.add(LSTM(100))
model.add(Dense(1,activation="sigmoid"))
model.compile(loss="binary_crossentropy",optimizer="adam",metrics=['accuracy'])

In [30]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 40)            400000    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               56400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 456,501
Trainable params: 456,501
Non-trainable params: 0
_________________________________________________________________


In [35]:
import numpy as np
X_final=np.array(embedded_docs)

In [36]:
Y_final=np.array(Y)

In [38]:
X_final.shape,Y_final.shape

((18285, 20), (18285,))

In [41]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X_final, Y_final, test_size=0.33, random_state=42)

## Model Training

In [43]:
#training our lstm model
model.fit(X_train,Y_train,validation_data=(X_test,Y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 25s 128ms/step - loss: 0.3304 - accuracy: 0.8438 - val_loss: 0.1922 - val_accuracy: 0.9205
Epoch 2/10
192/192 [==============================] - 27s 139ms/step - loss: 0.1232 - accuracy: 0.9528 - val_loss: 0.2109 - val_accuracy: 0.9228
Epoch 3/10
192/192 [==============================] - 28s 148ms/step - loss: 0.0654 - accuracy: 0.9773 - val_loss: 0.2282 - val_accuracy: 0.9137
Epoch 4/10
192/192 [==============================] - 28s 144ms/step - loss: 0.0251 - accuracy: 0.9921 - val_loss: 0.3536 - val_accuracy: 0.9176
Epoch 5/10
192/192 [==============================] - 28s 144ms/step - loss: 0.0103 - accuracy: 0.9974 - val_loss: 0.3738 - val_accuracy: 0.9135
Epoch 6/10
192/192 [==============================] - 23s 120ms/step - loss: 0.0061 - accuracy: 0.9987 - val_loss: 0.4645 - val_accuracy: 0.9167
Epoch 7/10
192/192 [==============================] - 18s 94ms/step - loss: 0.0032 - accuracy: 0.9993 - val_loss: 0.4250 - val_acc

## Performance Metrics and Accuracy

In [44]:
Y_pred=model.predict_classes(X_test)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


In [46]:
from sklearn.metrics import confusion_matrix

In [48]:
confusion_matrix(Y_test,Y_pred)

array([[3112,  307],
       [ 220, 2396]])

In [51]:

from sklearn.metrics import accuracy_score
accuracy_score(Y_test,Y_pred)

0.9126760563380282